<a href="https://colab.research.google.com/github/itisntsophie/vstavilytext/blob/main/vstavilytext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-telegram-bot --upgrade
!pip install nest_asyncio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 652.1/652.1 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.0 MB/s eta 0:00:00


In [ ]:
import nest_asyncio
import asyncio
from telegram import Update, Bot
from telegram.ext import Application, CommandHandler, MessageHandler, filters
import logging

# Логи для отладки
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)

# Токен вашего бота и ID канала для уведомлений
TOKEN = '7891089607:AAHD6SlsmQ7dTQMh5HZ3h8dQipbAuRmsB2M'
CHANNEL_ID = '@vstavilytext_1'  # Замените на ваш канал

# Разрешаем выполнение нескольких событийных циклов
nest_asyncio.apply()

# Функция, вызываемая при нажатии на /start
async def start(update: Update, context):
    await update.message.reply_text("привет, это мы, вставили текст! добавляем личные фотоистории в общий фотоальбом. отправь нам свою историю и фотографию!")

# Функция для обработки текстовых сообщений
async def handle_message(update: Update, context):
    user_message = update.message.text
    user = update.message.from_user
    bot = Bot(token=TOKEN)

    # Отправляем уведомление в канал
    notification = f"Пользователь @{user.username} отправил сообщение:\n{user_message}"
    await bot.send_message(chat_id=CHANNEL_ID, text=notification)

# Функция для обработки медиа файлов (фото)
async def handle_photo(update: Update, context):
    photo_file = await update.message.photo[-1].get_file()
    user = update.message.from_user
    bot = Bot(token=TOKEN)

    # Скачиваем фото
    await photo_file.download_to_drive('user_photo.jpg')

    # Получаем текст сообщения, если он есть
    caption = update.message.caption if update.message.caption else "Без подписи"

    # Отправляем уведомление в канал с фото и подписью
    notification = f"Пользователь @{user.username} отправил фото:\n{caption}"
    await bot.send_photo(chat_id=CHANNEL_ID, photo=open('user_photo.jpg', 'rb'), caption=notification)

# Основная функция
async def main():
    # Создание приложения
    application = Application.builder().token(TOKEN).build()

    # Обработчики команд и сообщений
    application.add_handler(CommandHandler('start', start))
    application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))
    application.add_handler(MessageHandler(filters.PHOTO, handle_photo))

    # Запуск бота
    await application.run_polling()

# Запуск основного кода через текущий цикл событий
asyncio.run(main())

RuntimeError: Cannot close a running event loop